In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import google.generativeai as genai

genai.configure()

### ReAct prompt + Few-shot prompting to enable in-context learning with gemini

In [3]:
from jinja2 import Environment, FileSystemLoader

PROMPTS_PATH = "prompts"
GEMINI_WIKIPEDIA_REACT_PROMPT = "gemini-react.jinja"

env = Environment(loader=FileSystemLoader(PROMPTS_PATH))
react_prompt_template = env.get_template(GEMINI_WIKIPEDIA_REACT_PROMPT)
react_prompt = react_prompt_template.render({"question": "xyz"})
print(react_prompt)

Solve a question answering task with interleaving Thought, Action, Observation steps. 
Thought can reason about the current situation, Observation is understanding relevant information from an Action's 
output and Action can be of three types:
(1) entity, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. 
    If not, it will return some similar entities to search and you can try to search the information from those topics.
(2) keyword, which returns the next sentence containing keyword in the current context. This only does exact matches, 
    so keep your searches short.
(3) answer, which returns the answer and finishes the task.

Here are some examples.

Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
Colorad

### The Gemini-ReAct pipeline

In [4]:
type(react_prompt_template)

jinja2.environment.Template

In [5]:
from jinja2.environment import Template

class ReAct:
    def __init__(self, model: str, react_prompt: Template):
        self.model = genai.GenerativeModel(model)
        self.chat = self.model.start_chat(history=[])
        self.should_continue_prompting = True
        self._search_history: list[str] = []
        self._search_urls: list[str] = []
        self._prompt = react_prompt

    @property
    def prompt(self):
        return self._prompt
    
    @classmethod
    def add_method(cls, func):
        setattr(cls, func.__name__, func)

    @staticmethod
    def clean(text: str):
        """Helper function for responses."""
        text = text.replace("\n", " ")
        return text


### Define tools

As instructed by the prompt, the model will be generating Thought-Action-Observation traces, 
where every Action trace could be one of the following tokens:

* </search/> : Perform a Wikipedia search via external API.
* </lookup/> : Lookup for specific information on a page with the Wikipedia API.
* </finish/> : Stop the execution of the model and return the answer.

If the model encounters any of these tokens, the model should make use of the tools made available
to the model. This understanding of the model to leverage acquired toolsets to collect information
from the external world is often referred to as function calling. Therefore, the next goal is to
imitate this function calling technique in order to allow ReAct prompted Gemini model to access
the external groundtruth.

The Gemini API supports function calling and you could use this feature to set up your tools. However, 
for this tutorial, you will learn to simulate it using stop_sequences parameter.


#### Search

Define a method to perform Wikipedia searches

In [6]:
import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError


@ReAct.add_method
def search(self, query: str):
    """Perfoms search on `query` via Wikipedia api and returns its summary.

    Args:
        query: Search parameter to query the Wikipedia API with.

    Returns:
        observation: Summary of Wikipedia search for `query` if found else
        similar search results.
    """
    observation = None
    query = query.strip()
    try:
        # try to get the summary for requested `query` from the Wikipedia
        observation = wikipedia.summary(query, sentences=4, auto_suggest=False)
        wiki_url = wikipedia.page(query, auto_suggest=False).url
        observation = self.clean(observation)

        # if successful, return the first 2-3 sentences from the summary as model's context
        observation = self.model.generate_content(
            f"Retun the first 2 or 3 \
      sentences from the following text: {observation}"
        )
        observation = observation.text

        # keep track of the model's search history
        self._search_history.append(query)
        self._search_urls.append(wiki_url)
        print(f"Information Source: {wiki_url}")

    # if the page is ambiguous/does not exist, return similar search phrases for model's context
    except (DisambiguationError, PageError) as e:
        observation = f'Could not find ["{query}"].'
        # get a list of similar search topics
        search_results = wikipedia.search(query)
        observation += (
            f" Similar: {search_results}. You should search for one of those instead."
        )

    return observation

### Lookup

In [7]:

@ReAct.add_method
def lookup(self, phrase: str, context_length=200):
    """Searches for the `phrase` in the lastest Wikipedia search page
    and returns number of sentences which is controlled by the
    `context_length` parameter.

    Args:
        phrase: Lookup phrase to search for within a page. Generally
        attributes to some specification of any topic.

        context_length: Number of words to consider
        while looking for the answer.

    Returns:
        result: Context related to the `phrase` within the page.
    """
    # get the last searched Wikipedia page and find `phrase` in it.
    page = wikipedia.page(self._search_history[-1], auto_suggest=False)
    page = page.content
    page = self.clean(page)
    start_index = page.find(phrase)

    # extract sentences considering the context length defined
    result = page[max(0, start_index - context_length):start_index+len(phrase)+context_length]
    print(f"Information Source: {self._search_urls[-1]}")
    return result

#### Finish

In [8]:
@ReAct.add_method
def finish(self, _):
    """Finishes the conversation on encountering  token by
    setting the `self.should_continue_prompting` flag to `False`.
    """
    self.should_continue_prompting = False
    print(f"Information Sources: {self._search_urls}")

### Stop tokens and function calling imitation
Now that you are all set with function definitions, the next step is to instruct the model to interrupt its execution upon encountering any of the action tokens. You will make use of the stop_sequences parameter from genai.GenerativeModel.GenerationConfig class to instruct the model when to stop. Upon encountering an action token, the pipeline will simply extract what specific token from the stop_sequences argument terminated the model's execution, and then call the appropriate tool (function).

The function's response will be added to model's chat history for continuing the context link.

In [9]:
import re


@ReAct.add_method
def __call__(self, user_question, max_calls: int = 8, **generation_kwargs):
    """Starts multi-turn conversation with the chat models with function calling

    Args:
        max_calls: max calls made to the model to get the final answer.

        generation_kwargs: Same as genai.GenerativeModel.GenerationConfig
                candidate_count: (int | None) = None,
                stop_sequences: (Iterable[str] | None) = None,
                max_output_tokens: (int | None) = None,
                temperature: (float | None) = None,
                top_p: (float | None) = None,
                top_k: (int | None) = None

    Raises:
        AssertionError: if max_calls is not between 1 and 8
    """

    # hyperparameter fine-tuned according to the paper
    assert 0 < max_calls <= 8, "max_calls must be between 1 and 8"

    if len(self.chat.history) == 0:
        model_prompt = self.prompt.render({"question": user_question})
    else:
        model_prompt = user_question

    print("model_prompt -> ", model_prompt)

    # stop_sequences for the model to immitate function calling
    callable_entities = ["", "", ""]

    generation_kwargs.update({"stop_sequences": callable_entities})

    self.should_continue_prompting = True
    for idx in range(max_calls):

        self.response = self.chat.send_message(
            content=[model_prompt], generation_config=generation_kwargs, stream=False
        )

        for chunk in self.response:
            print(chunk.text, end=" ")

        response_cmd = self.chat.history[-1].parts[-1].text

        try:
            # regex to extract
            cmd = re.findall(r"<(.*)>", response_cmd)[-1]
            print(f"{cmd}>")
            # regex to extract param
            query = response_cmd.split(f"<{cmd}>")[-1].strip()
            # call to appropriate function
            observation = self.__getattribute__(cmd)(query)

            if not self.should_continue_prompting:
                break

            stream_message = f"\nObservation {idx + 1}\n{observation}"
            print(stream_message)
            # send function's output as user's response
            model_prompt = f"<{cmd}>{query}{cmd}>'s Output: {stream_message}"

        except (IndexError, AttributeError) as e:
            model_prompt = "Please try to generate thought-action-observation traces \
      as instructed by the prompt."

### Test ReAct prompted Gemini model

In [10]:
gemini_react_chat = ReAct(model='gemini-1.5-flash', react_prompt=react_prompt_template)
# Note: try different combinations of generational_config parameters for variational results
gemini_react_chat("What are the total of ages of the main trio from the new Percy Jackson and the Olympians TV series in real life?", temperature=0.2)

model_prompt ->  Solve a question answering task with interleaving Thought, Action, Observation steps. 
Thought can reason about the current situation, Observation is understanding relevant information from an Action's 
output and Action can be of three types:
(1) entity, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. 
    If not, it will return some similar entities to search and you can try to search the information from those topics.
(2) keyword, which returns the next sentence containing keyword in the current context. This only does exact matches, 
    so keep your searches short.
(3) answer, which returns the answer and finishes the task.

Here are some examples.

Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
